In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.integrate as spi
from scipy.optimize import curve_fit,minimize,fmin
import numpy as np
import matplotlib 
import optuna
import pickle as pkl  
np.set_printoptions(suppress=True)
plt.rc('text', usetex=True)

dataDir = "/home/polivares/Dropbox/DocUSM/Researches/InfluenzaMeningococcal/Data/"
imageDir = "/home/polivares/Dropbox/DocUSM/Researches/InfluenzaMeningococcal/Scripts/images/"

In [2]:
# Loading data
menSeries = pd.read_pickle(dataDir + "pkls/menSeries.pkl")
fluSeries = pd.read_pickle(dataDir + "pkls/fluSeries.pkl")
ausPop = pd.read_pickle(dataDir + "pkls/ausPop.pkl")

In [3]:
menSeries

date
2009-01    25
2009-02     7
2009-03    15
2009-04    18
2009-05    17
           ..
2017-08    39
2017-09    67
2017-10    51
2017-11    28
2017-12    23
Name: Men, Length: 108, dtype: int64

In [4]:
import os
if os.path.isfile(dataDir + "flu_menSIIRParams.pkl"):
    flu_menParams = pkl.load(open(dataDir + "flu_menSIIRParams.pkl", "rb"))
else:
    flu_menParams = {}
    
menParams = pkl.load(open(dataDir + "pkls/menSIRParams.pkl", "rb"))
fluParams = pkl.load(open(dataDir + "pkls/fluSIRParams.pkl", "rb"))

In [5]:
print(menParams)
print(fluParams)

{2017: [69.37033397309656, 69.30257707258437, 0.1516317620776813], 2009: [60.231846774940124, 60.21980222020443, 0.15275229894586662], 2010: [-0.007996924771336527, 1.5674723254362843e-05, 0.10309441643286982], 2011: [-0.0012161035454762803, 0.04575438032812004, 0.10341005926078485], 2012: [168.30591038787196, 168.16297383533924, 0.13257514233058662], 2013: [121.34425185049055, 121.29626573215431, 0.1530242986181454], 2014: [212.8582713786913, 212.6516034393233, 0.045270446046769275], 2015: [254.14954418266785, 253.87905864700417, 0.0650289278363755], 2016: [-0.0019203871490743678, -0.020079890030061442, 0.12449161250375251]}
{2017: [8.993779282533158, 8.347865953418331, 0.8051828650381628], 2008: [42.5326377333672, 41.95061087323178, 0.6177425164247533], 2009: [23.119115425141842, 22.083991154996014, 0.7438191626954874], 2010: [32.30813365036235, 31.80694514966924, 0.5097104487520645], 2011: [22.91502211555694, 22.474019634990025, 0.40824883608999374], 2012: [24.891229614896744, 24.09

In [6]:
# Evaluation time
from datetime import datetime # Library for datetime format
from dateutil import relativedelta # Library to calculate delta time from date

year = 2017
startdate = str(year) + '-01'
enddate = str(year) + '-12'


date_format = "%Y-%m"
sd = datetime.strptime(startdate, date_format)
ed = datetime.strptime(enddate, date_format)

n_months = relativedelta.relativedelta(ed, sd) # Number of months (delta time)
print(n_months)

relativedelta(months=+11)


In [7]:
# Timestamp parameters
t_start = 0.0; t_end = n_months.months; t_inc = 1
t_range = np.arange(t_start, t_end+t_inc, t_inc)


In [8]:
# Initial conditions
SIIR0 = np.zeros(9)
SIIR0[1] = (fluSeries[startdate])
SIIR0[2] = (menSeries[startdate])
SIIR0[0] = ausPop[year] - np.sum(SIIR0[1:8])

fluMenSeries = np.concatenate((fluSeries[startdate:enddate].values,menSeries[startdate:enddate].values))

In [9]:
# Defining SIIR isolated equations
def SIIR_eqs(SIIR0,t,beta1,beta2, delta1, delta2, beta1prime, beta2prime, delta1prime, delta2prime):
    #beta1, beta2, delta1, delta2, beta1prime, beta2prime, delta1prime, delta2prime = params
    SSi, ISi, SIi, IIi, RSi, SRi, RIi, IRi, RRi = SIIR0
    
    N = np.sum(SIIR0)
    
    SS = -SSi * beta1 * (ISi + IIi + IRi) / N - SSi * beta2 * (SIi + IIi + RIi) / N
    SI = beta2*SSi*(SIi + IIi + RIi)/N - beta1prime*SIi*(ISi + IIi + IRi)/N - delta2*SIi 
    SR = delta2 * SIi - SRi * beta1 * (ISi + IIi + IRi)/N
    IS = SSi * beta1 * (ISi + IIi + IRi) / N - delta1 * ISi - ISi * beta2prime * (SIi + IIi + RIi) / N
    II = ISi * beta2prime * (SIi + IIi + RIi) / N + SIi * beta1prime * (ISi + IIi + IRi) / N - delta1prime * IIi - delta2prime * IIi
    IR = SRi * beta1 * (ISi + IIi + IRi) / N + delta2prime * IIi - delta1 * IRi
    RS = delta1 * ISi - RSi * beta2 * (SIi + IIi + RIi) / N
    RI = RSi * beta2 * (SIi + IIi + RIi) / N + delta1prime * IIi - delta2 * RIi
    RR = delta1 * IRi + delta2 * RIi

    return SS, IS, SI, II, RS, SR, RI, IR, RR

# Fitting function from infected data and I state on SIR model
def fitSIIR(t, beta1,beta2, gamma1, gamma2, beta1int, beta2int, gamma1int, gamma2int):
    sir_res = spi.odeint(SIIR_eqs,SIIR0,t,args=(beta1,beta2, gamma1, gamma2, beta1int, beta2int, gamma1int, gamma2int)) 
    I1 = sir_res[:,1] + sir_res[:,3] + sir_res[:,7] 
    I2 = sir_res[:,2] + sir_res[:,3] + sir_res[:,6]
    return I1,I2

def fitErrorSIIR(trial):
    #(9.00165750105233, 69.35210027387456, 8.35506567120736, 69.35957442372714, 10.060360325775953, 97.56183004648915, 4.957084610524714, 53.932494914485716)
    #beta1 = trial.suggest_float("beta1",8,10,step=0.0001)
    beta1 = trial.suggest_float("beta1",1,100,step=0.0001)
    #beta2 = trial.suggest_float("beta2",68,71,step=0.0001)
    beta2 = trial.suggest_float("beta2",1,100,step=0.0001)
    #gamma1 = trial.suggest_float("gamma1",7,9,step=0.0001)
    gamma1 = trial.suggest_float("gamma1",1,100,step=0.0001)
    #gamma2 = trial.suggest_float("gamma2",68,71,step=0.0001)
    gamma2 = trial.suggest_float("gamma2",1,100,step=0.0001)
    #beta1int = trial.suggest_float("beta1int",9,11,step=0.0001)
    beta1int = trial.suggest_float("beta1int",1,150,step=0.0001)
    #beta2int = trial.suggest_float("beta2int",96,101,step=0.0001)
    beta2int = trial.suggest_float("beta2int",1,150,step=0.0001)
    #gamma1int = trial.suggest_float("gamma1int",4,6,step=0.0001)
    gamma1int = trial.suggest_float("gamma1int",1,150,step=0.0001)
    #gamma2int = trial.suggest_float("gamma2int",52,54,step=0.0001)
    gamma2int = trial.suggest_float("gamma2int",1,150,step=0.0001)
    #w = trial.suggest_float("w",1,100)
    w=10000
    params = beta1,beta2, gamma1, gamma2, beta1int, beta2int, gamma1int, gamma2int

    #beta1,beta2, gamma1, gamma2, beta1int, beta2int, gamma1int, gamma2int = params
    def normMSE(data,model):
        n = len(data)
        mse = ((data-model)**2).sum()
        nmse = mse/(n*(data.sum()/n)*(model.sum()/n))
        
        return nmse
    params_np = np.array(params)
    k = 1 # penalization
    if np.any(params_np<0):
        k = 1000000
    I1,I2 = fitSIIR(t_range, beta1,beta2, gamma1, gamma2, beta1int, beta2int, gamma1int, gamma2int)
    sim = [normMSE(fluSeries[startdate:enddate],I1),normMSE(menSeries[startdate:enddate],I2)]   
    return k * np.linalg.norm(sim)
    

In [10]:
len(I1)

NameError: name 'I1' is not defined

In [ ]:
len(np.concatenate((I1,I2)))

In [12]:
study = optuna.create_study()
study.optimize(fitErrorSIIR, n_trials=10000)

study.best_params  # E.g. {'x': 2.002108042}

4.084, 'gamma1': 10.2277, 'gamma2': 17.8098, 'beta1int': 35.8035, 'beta2int': 58.9012, 'gamma1int': 59.451800000000006, 'gamma2int': 144.81560000000002}. Best is trial 1824 with value: 4.846826106988934.
[I 2021-03-30 02:46:57,158] Trial 9942 finished with value: 90.63942538382052 and parameters: {'beta1': 10.2209, 'beta2': 52.668600000000005, 'gamma1': 7.1943, 'gamma2': 19.703, 'beta1int': 30.485400000000002, 'beta2int': 75.7458, 'gamma1int': 69.59, 'gamma2int': 143.0447}. Best is trial 1824 with value: 4.846826106988934.
[I 2021-03-30 02:46:58,142] Trial 9943 finished with value: 60.10968141734645 and parameters: {'beta1': 4.6006, 'beta2': 64.1856, 'gamma1': 2.9131, 'gamma2': 21.1571, 'beta1int': 33.708800000000004, 'beta2int': 65.37610000000001, 'gamma1int': 65.2829, 'gamma2int': 141.3836}. Best is trial 1824 with value: 4.846826106988934.
[I 2021-03-30 02:46:59,046] Trial 9944 finished with value: 253.6189607953796 and parameters: {'beta1': 6.8239, 'beta2': 68.0852, 'gamma1': 8.498

{'beta1': 6.3705,
 'beta2': 71.91550000000001,
 'gamma1': 12.092,
 'gamma2': 18.3091,
 'beta1int': 16.3675,
 'beta2int': 134.81300000000002,
 'gamma1int': 92.7578,
 'gamma2int': 129.793}

In [ ]:
"""p0=[fluParams[year][0],menParams[year][0],fluParams[year][1],menParams[year][1],
    fluParams[year][0],menParams[year][0],fluParams[year][1],menParams[year][1]]

#COBYLA
#params = minimize(fitErrorSIIR,p0,options={'maxiter':10000},
#        bounds=((0,None),(0,None),(0,None),(0,None),
#               (0,None),(0,None),(0,None),(0,None)))

answ=fmin(fitErrorSIIR,p0,full_output=1,maxiter=10000)
params=answ[0]
bestscore=answ[1]
print(params,bestscore)
"""

In [ ]:
#beta1,beta2, gamma1, gamma2, beta1int, beta2int, gamma1int, gamma2int = params
beta1,beta2, gamma1, gamma2, beta1int, beta2int, gamma1int, gamma2int= study.best_params.values()

In [ ]:
flu_menParams[year] = [beta1,beta2, gamma1, gamma2, beta1int, beta2int, gamma1int, gamma2int]
pkl.dump(flu_menParams, open(dataDir + 'pkls/flu_menSIIRParams.pkl','wb'))

# Evaluation 

In [ ]:
# Defining SIR isolated equations
def SIR_eqs(SIR0,t, beta, gamma):
    S0=SIR0[0]
    I0=SIR0[1]
    R0=SIR0[2]

    S = - beta * S0 * I0/ausPop[year]
    I = (beta * S0 * I0/ausPop[year]) - gamma * I0
    R = gamma * I0

    return (S,I,R)

# Fitting function from infected data and I state on SIR model
def fitSIR(t, beta, gamma):
    return spi.odeint(SIR_eqs,SIR0,t_range,args=(beta,gamma))[:,1] 

## Influenza (isolated)

In [ ]:
# Timestamp parameters
t_start = 0.0; t_end = n_months.months; t_inc = 0.01
t_eval = np.arange(t_start, t_end+t_inc, t_inc)

# Initial conditions
S0 = (ausPop[year] - fluSeries[startdate])
I0 = (fluSeries[startdate])
R0 = 0

SIR0 = [S0,I0,R0]

In [ ]:
def fluSIRSim(beta,gamma):    
    return spi.odeint(SIR_eqs,SIR0,t_eval,args=(beta,gamma))

In [ ]:
betaFlu = fluParams[year][0]
gammaFlu = fluParams[year][1]
SIRflu = fluSIRSim(betaFlu,gammaFlu)
Sflu = SIRflu[:,0]
Iflu = SIRflu[:,1]
Rflu = SIRflu[:,2]

In [ ]:
plt.plot(t_range[:n_months.months+1], fluSeries[startdate:enddate].values,'ok',label="Original data")
plt.plot(t_eval,Iflu,'-r',label="Infected fit")
plt.show()
plt.close()

## Meningococcal (isolated)

In [ ]:
# Timestamp parameters
t_start = 0.0; t_end = n_months.months; t_inc = 0.01
t_eval = np.arange(t_start, t_end+t_inc, t_inc)

# Initial conditions
S0 = (ausPop[year] - menSeries[startdate])
I0 = (menSeries[startdate])
R0 = 0

SIR0 = [S0,I0,R0]

In [ ]:
def menSIRSim(beta,gamma):    
    return spi.odeint(SIR_eqs,SIR0,t_eval,args=(beta,gamma))

In [ ]:
betaMen = menParams[year][0]
gammaMen = menParams[year][1]

SIRmen = menSIRSim(betaMen,gammaMen)
Smen = SIRmen[:,0]
Imen = SIRmen[:,1]
Rmen = SIRmen[:,2]

In [ ]:

plt.plot(t_range[:n_months.months+1], menSeries[startdate:enddate].values,'ok',label="Original data")
plt.plot(t_eval,Imen,'-r',label="Infected fit")
plt.text(15, 60, r'$\gamma = $' + str(gammaMen))
plt.text(15, 64, r'$\beta = $' + str(betaMen))
plt.show()
plt.close()

## Influenza / Meningococcal (interactive)

In [ ]:
# Timestamp parameters
t_start = 0.0; t_end = n_months.months; t_inc = 0.01
t_eval = np.arange(t_start, t_end+t_inc, t_inc)

# Initial conditions
SIIR0 = np.zeros(9)
SIIR0[1] = (fluSeries[startdate])
SIIR0[2] = (menSeries[startdate])
SIIR0[0] = ausPop[year] - np.sum(SIIR0[1:8])

In [ ]:
def menfluSIIRSim(beta1,beta2,gamma1, gamma2, beta1int, beta2int, gamma1int, gamma2int):    
    return spi.odeint(SIIR_eqs,SIIR0,t_eval,args=(beta1,beta2, gamma1, gamma2, beta1int, beta2int, gamma1int, gamma2int)) 

In [ ]:
SIIR = menfluSIIRSim(beta1,beta2,gamma1, gamma2, beta1int, beta2int, gamma1int, gamma2int)
I1 = SIIR[:,1] + SIIR[:,3] + SIIR[:,7] 
I2 = SIIR[:,2] + SIIR[:,3] + SIIR[:,6]
 

In [ ]:
# Influenza

plt.plot(t_range[:n_months.months+1], fluSeries[startdate:enddate].values,'ok',label="Original data")
plt.plot(t_eval,Iflu,'-r',label="Isolated infected fit")
plt.plot(t_eval,I1,'-b',label="Interactive infected fit")
plt.legend()
plt.title("Influenza Year: " + str(year))
plt.savefig(imageDir + "flu_menInteractive/fluInteractive_" + str(year) + ".png" )
plt.show()
plt.close()

In [ ]:
# Meningococcal
plt.plot(t_range[:n_months.months+1], menSeries[startdate:enddate].values,'ok',label="Original data")
plt.plot(t_eval,Imen,'-r',label="Isolated infected fit")
plt.plot(t_eval,I2,'-b',label="Interactive infected fit")
plt.legend()
plt.title("Meningococcal Year: " + str(year))
plt.savefig(imageDir + "flu_menInteractive/menInteractive_" + str(year) + ".png" )
plt.show()
plt.close()

In [ ]:
params = (beta1,beta2,gamma1,gamma2,beta1int,beta2int,gamma1int, gamma2int)
print(params)
print(betaFlu,betaMen,gammaFlu,gammaMen)